# **Neighborhood Ranking - Buffalo,NY | Third Estate Analytics**

## Overview about the company 

##### Third estate analytics is a company owned by Third Estate Ventures which is a real estate agency in the Buffalo/Niagara region. The organisation is developing a tool which can be used to suggest neighborhoods based on the need and amount of development it had over years. They are interested in investing on the fastest developing regions in all the similar sized cities.  

## Problem statement 

The client is interested in investing in the regions which might develop in the forthcoming years. The overall idea of the project is to build a model which can determine the ranking of all the neighborhoods across the Buffalo in order of development over years .The broader scope of the project is to run the model across all the similar sized cities across the US . 

***Dataset*** : To achieve the above goal the datasets are being taken from Open data portal of Buffalo .
URL : https://data.buffalony.gov/
The datasets considered are :
1. Tows 
2. Parking Violations
3. Code Violations
4. Assessment Roll 
5. Housing Violations 

### Number of Records - 2.5 million rows

The datasets are mapped against walkscore and assessement value

***Walkscore***:Walk Score measures the walkability of any address using a patented system. Walk Score also measures pedestrian friendliness by analyzing population density and road metrics such as block length and intersection density

***Assessed Value***:An assessed value is the dollar value assigned to a property to measure applicable taxes. Assessed valuation determines the value of a residence for tax purposes and takes comparable home sales and inspections into consideration

**Libraries Used**: Pandas,numpy,seaborn,matplotlib,scipy,sklearn,tensorflow

**Machine Learning Models**

Random Forest Regressor and classifier

Gradient boosting Regressor

### Importing the required libaries for Exploratory data analysis and Machine learning

In [0]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.ensemble import GradientBoostingRegressor
from google.colab import files
from IPython.display import Image

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Mounting the google drive to google colab notebook

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Connecting and using GPU 

In [0]:
import tensorflow as tf 
tf.test.gpu_device_name()

'/device:GPU:0'

### Reading the parking summonses data from google drive 

### Years considered - (2008 - 2019)

In [0]:
parking_summonses=pd.read_csv('/content/drive/My Drive/Gentrification/Parking_Summonses .csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
parking_summonses['CENSUS TRACT'].value_counts()

165      222531
165.0    102745
68       101603
169       60707
63.01     54966
          ...  
44.02       552
46.02       199
46.02        55
62.01        19
62.01        12
Name: CENSUS TRACT, Length: 159, dtype: int64

In [0]:
parking_summonses.shape

(2155859, 18)

### Reading the census tracts and corresponding neigborhood data from google drive

In [0]:
tracts_data=pd.read_csv('/content/drive/My Drive/Gentrification/NEighborhoods and tracts.csv')

In [0]:
parking_summonses['CENSUS TRACT'] = pd.to_numeric(parking_summonses['CENSUS TRACT'], errors='coerce')
parking_summonses = parking_summonses.dropna(subset=['CENSUS TRACT'])
parking_summonses['CENSUS TRACT']=parking_summonses['CENSUS TRACT'].astype(float)

In [0]:
tracts_data['CENSUS TRACT'] = pd.to_numeric(tracts_data['CENSUS TRACT'], errors='coerce')
tracts_data = tracts_data.dropna(subset=['CENSUS TRACT'])
tracts_data['CENSUS TRACT']=tracts_data['CENSUS TRACT'].astype(float)

### Reading the Neighborhood and corresponding walkscore data from google drive

In [0]:
score=pd.read_excel('/content/drive/My Drive/Gentrification/Walkscore_final.xlsx')

In [0]:
score.columns

Index(['NEIGHBORHOOD', 'WALKSCORE', 'RANK', 'GENTRIFICATION SCORE',
       'CLASSIFCATION ', 'CLASS'],
      dtype='object')

### Mapping the corresponding Parking violations to the respective neighborhoods using the common attribute(Census Tract)

In [0]:
final=parking_summonses.merge(tracts_data,on='CENSUS TRACT')

### Mapping the corresponding Neighborhood with the Walkscore using the common attribute of both tables (Neighborhood)

In [0]:
parking_final_1=final.merge(score,on='NEIGHBORHOOD')

### Sample data frame after merging all the three tables
###1. Parking Summonses
###2. Neighborhoods and tracts
###3. Neighborhoods and Walkscores

In [0]:
parking_final_1.head()

,SUMMONS NUMBER,SUMMONS DATE,VIOLATION TIME,VIOLATION DESCRIPTION,ORIGINAL FINE,VIOLATION ADDRESS NUMBER,VIOLATION STREET,VIOLATION FULL ADDRESS,CITY,STATE,LATITUDE,LONGITUDE,LOCATION,COUNCIL DISTRICT,POLICE DISTRICT,CENSUS TRACT,CENSUS BLOCK GROUP,CENSUS BLOCK,NEIGHBORHOOD,WALKSCORE,RANK,GENTRIFICATION SCORE,CLASSIFCATION,CLASS
0,X2308148,08/02/2019,0125P,PARKING VIOLATION,40.0,175.0,FRANKLIN,175 FRANKLIN,Buffalo,NY,42.888195,-78.875835,"(42.8881947942094, -78.87583500193372)",ELLICOTT,District B,165.0,1,1028,Central,55,36,65,25,D
1,X2308149,08/02/2019,0130P,NO STANDING,40.0,37.0,HURON WEST,37 HURON WEST,Buffalo,NY,42.888948,-78.874872,"(42.88894814599496, -78.87487202385579)",ELLICOTT,District B,165.0,1,1022,Central,55,36,65,25,D
2,X2308151,08/02/2019,0305P,METER OVERTIME,40.0,228.0,FRANKLIN,228 FRANKLIN,Buffalo,NY,42.890040,-78.875395,"(42.89003992129964, -78.87539503306608)",ELLICOTT,District B,165.0,1,1023,Central,55,36,65,25,D
3,X2308152,08/02/2019,0312P,METER OVERTIME,40.0,210.0,FRANKLIN,210 FRANKLIN,Buffalo,NY,42.889319,-78.875628,"(42.88931932925486, -78.87562789846032)",ELLICOTT,District B,165.0,1,1023,Central,55,36,65,25,D
4,X2308153,08/02/2019,0320P,METER OVERTIME,40.0,22.0,SENECA WEST,22 SENECA WEST,Buffalo,NY,42.880596,-78.875566,"(42.880595539253534, -78.87556606949188)",FILLMORE,District B,165.0,1,1114,Central,55,36,65,25,D


### Displaying the number of violations in each neighborhood over the years 

In [0]:
parking_final_1['NEIGHBORHOOD'].value_counts()

Elmwood Bidwell       354553
Central               325276
Elmwood Bryant        208101
Fruit Belt            193712
North Park            162378
Allentown             153824
Broadway Fillmore     131946
Lower West Side       120374
University Heights     84124
Masten Park            81852
Seneca-Cazenovia       79536
West Side              72960
South Park             69489
Upper West Side        68069
Hamlin Park            67342
Parkside               66049
Central Park           55958
First Ward             46748
Kenfield               44992
Black Rock             43344
Kensington-Bailey      42517
Fillmore-Leroy         40713
Lovejoy                38723
Riverside              38019
Genesee-Moselle        31553
Schiller Park          28356
Hopkins-Tifft          22245
Grant-Amherst          21113
Seneca Babcock         19906
Kaisertown             19769
Delavan Grider         18925
Pratt-Willert          17163
MLK Park               15597
Ellicott               15281
West Hertel   

In [0]:
parking_final_1['CENSUS TRACT'].value_counts()

165.00    325276
31.00     252786
68.00     153824
63.01      98509
71.02      92218
           ...  
28.00       4983
14.02       4569
29.00       4303
15.00       3216
39.01       2091
Name: CENSUS TRACT, Length: 76, dtype: int64

In [0]:
parking_final_1.dropna()

,SUMMONS NUMBER,SUMMONS DATE,VIOLATION TIME,VIOLATION DESCRIPTION,ORIGINAL FINE,VIOLATION ADDRESS NUMBER,VIOLATION STREET,VIOLATION FULL ADDRESS,CITY,STATE,LATITUDE,LONGITUDE,LOCATION,COUNCIL DISTRICT,POLICE DISTRICT,CENSUS TRACT,CENSUS BLOCK GROUP,CENSUS BLOCK,NEIGHBORHOOD,WALKSCORE,RANK,GENTRIFICATION SCORE,CLASSIFCATION,CLASS
0,X2308148,08/02/2019,0125P,PARKING VIOLATION,40.0,175.0,FRANKLIN,175 FRANKLIN,Buffalo,NY,42.888195,-78.875835,"(42.8881947942094, -78.87583500193372)",ELLICOTT,District B,165.0,1,1028,Central,55,36,65,25,D
1,X2308149,08/02/2019,0130P,NO STANDING,40.0,37.0,HURON WEST,37 HURON WEST,Buffalo,NY,42.888948,-78.874872,"(42.88894814599496, -78.87487202385579)",ELLICOTT,District B,165.0,1,1022,Central,55,36,65,25,D
2,X2308151,08/02/2019,0305P,METER OVERTIME,40.0,228.0,FRANKLIN,228 FRANKLIN,Buffalo,NY,42.890040,-78.875395,"(42.89003992129964, -78.87539503306608)",ELLICOTT,District B,165.0,1,1023,Central,55,36,65,25,D
3,X2308152,08/02/2019,0312P,METER OVERTIME,40.0,210.0,FRANKLIN,210 FRANKLIN,Buffalo,NY,42.889319,-78.875628,"(42.88931932925486, -78.87562789846032)",ELLICOTT,District B,165.0,1,1023,Central,55,36,65,25,D
4,X2308153,08/02/2019,0320P,METER OVERTIME,40.0,22.0,SENECA WEST,22 SENECA WEST,Buffalo,NY,42.880596,-78.875566,"(42.880595539253534, -78.87556606949188)",FILLMORE,District B,165.0,1,1114,Central,55,36,65,25,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812642,Z2222894,01/29/2019,0836A,ALTERNATE PARKING,40.0,36.0,GREELEY,36 GREELEY,Buffalo,NY,42.948733,-78.882463,"(42.948732632109, -78.882463304971)",NORTH,District D,56.0,5,5004,West Hertel,73,16,85,75,B
2812643,Z2222895,01/29/2019,0839A,ALTERNATE PARKING,40.0,55.0,HOLMES,55 HOLMES,Buffalo,NY,42.949007,-78.887397,"(42.949006902999, -78.887396920712)",NORTH,District D,56.0,4,4000,West Hertel,73,16,85,75,B
2812644,Z2239935,03/20/2019,1217P,NO PARKING,40.0,44.0,SAYRE,44 SAYRE,Buffalo,NY,42.947758,-78.893552,"(42.94775780857678, -78.89355160442582)",NORTH,District D,56.0,2,2004,West Hertel,73,16,85,75,B
2812645,Z2248919,03/19/2019,0832A,NO PARKING,40.0,152.0,LAWN,152 LAWN,Buffalo,NY,42.949771,-78.887316,"(42.949771006866236, -78.88731610640603)",NORTH,District D,56.0,4,4000,West Hertel,73,16,85,75,B


### String manipulations to fetch the years

In [0]:
parking_final_1['SUMMONS DATE']=parking_final_1['SUMMONS DATE'].apply(lambda x:x[6] + x[7] +x[8] +x[9]) 
parking_final_1['SUMMONS DATE']=parking_final_1['SUMMONS DATE'].apply(lambda x:float(x))

### Excluding the year 2019 because the dataset is incomplete for 2019 and might lead to wrong conclusions

In [0]:
parking_final_1=parking_final_1[parking_final_1['SUMMONS DATE'] != '2019.0']

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Creating an object for label binarizor to label encode all the categorical values of the column 'Violation Description'
### ***Label Encoding***: Label encoding is a technique which allows you to convert each value in a column to a number. Numerical labels are always between 0 and n_categories-1.

In [0]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb_results = lb.fit_transform(parking_final_1['VIOLATION DESCRIPTION'])
lb_results_df = pd.DataFrame(lb_results, columns=lb.classes_)

print(lb_results_df.head())

   ALTERNATE PARKING  ...  VEHICLE PARKED ON UNPAVED PORTION OR
0                  0  ...                                     0
1                  0  ...                                     0
2                  0  ...                                     0
3                  0  ...                                     0
4                  0  ...                                     0

[5 rows x 39 columns]


In [0]:
parking_final2 = pd.concat([parking_final_1, lb_results_df], axis=1)

print(parking_final2.head())

  SUMMONS NUMBER  ...  VEHICLE PARKED ON UNPAVED PORTION OR
0       X2308148  ...                                     0
1       X2308149  ...                                     0
2       X2308151  ...                                     0
3       X2308152  ...                                     0
4       X2308153  ...                                     0

[5 rows x 63 columns]


### Label Encoding the years to get an idea about the number of violations increasing every year 

In [0]:

lb = LabelBinarizer()
lb_results1 = lb.fit_transform(parking_final2['SUMMONS DATE'])
lb_results_df1 = pd.DataFrame(lb_results1, columns=lb.classes_)

print(lb_results_df.head())

   ALTERNATE PARKING  ...  VEHICLE PARKED ON UNPAVED PORTION OR
0                  0  ...                                     0
1                  0  ...                                     0
2                  0  ...                                     0
3                  0  ...                                     0
4                  0  ...                                     0

[5 rows x 39 columns]


In [0]:
parking_final = pd.concat([parking_final2, lb_results_df1], axis=1)

print(parking_final.head())

  SUMMONS NUMBER  SUMMONS DATE VIOLATION TIME  ... 2017.0  2018.0  2019.0
0       X2308148        2019.0          0125P  ...      0       0       1
1       X2308149        2019.0          0130P  ...      0       0       1
2       X2308151        2019.0          0305P  ...      0       0       1
3       X2308152        2019.0          0312P  ...      0       0       1
4       X2308153        2019.0          0320P  ...      0       0       1

[5 rows x 76 columns]


In [0]:
 parking_final['RANK'].fillna((parking_final['RANK'].mean()),inplace=True)

### The following are the columns considered as feature inputs to the Machine learning model. The number of columns have increased after label encoding considering each violation as a seperate feature

In [0]:
feature_cols=['ALTERNATE PARKING',
                             'BLOCKED DRIVEWAY (ALL)',
       'BUS RT. NO PARKING 1:30 TO 7:00 AM 11/15-4/1',
                                     'DOUBLE PARKING',
                   'EXPIRED INSPECTION STICKER (ALL)',
                       'EXPIRED REGISTRATION STICKER',
                     'LESS THAN 20 FT FROM CROSSWALK',
                                       'LOADING ZONE',
                                     'METER OVERTIME',
                             'METHOD OF PARKING(ALL)',
                       'NO INSPECTIONS STICKER (ALL)',
             'NO PARK COMMERICAL TRCK ON RESIDENTIAL',
                                         'NO PARKING',
                 'NO PARKING OF HOUSE TRAILERS (ALL)',
                      'NO REGISTRATION STICKER (ALL)',
              'NO REPAIRS IN STREET EXCEPT EMERGENCY',
                                        'NO STANDING',
                                  'NO STOPPING (ALL)',
                                       'NO VIOLATION',
                    'ONE LICENSE PLATE MISSING (ALL)',
                    'PARKED BEYOND POSTED TIME LIMIT',
                         'PARKED IN HANDICAPPED AREA',
              'PARKED LESS THAN 15 FT - FIRE HYDRANT',
                      'PARKED MORE THAN 12"FROM CURB',
                 'PARKED MORE THAN 72 HRS. SAME SPOT',
                      'PARKED ON BRIDGE OR IN SUBWAY',
                           'PARKED ON SIDEWALK (ALL)',
                'PARKED OVER 18 FT FOR MORE THAN ONE',
                   'PARKING IN WRONG DIRECTION (ALL)',
                 'PARKING ON DIVIDED HWYS PROHIBITED',
           'PARKING PROHIBITED IN CONSTRUCTION SITES',
                                  'PARKING VIOLATION',
                      'PARKING WITHOUT CONSENT (ALL)',
                  'PERSISTEN VIOLATORS AND SCOFFLAWS',
                              'SWITCHED PLATES (ALL)',
                                  'UNKNOWN VIOLATION',
               'UNREGISTERED VEH.ON PRIVATE PROPERTY',
                 'VEHICLE BLOCKING HANDICAP CURB CUT',
               'VEHICLE PARKED ON UNPAVED PORTION OR',
                                               2007.0,
                                               2008.0,
                                               2009.0,
                                               2010.0,
                                               2011.0,
                                               2012.0,
                                               2013.0,
                                               2014.0,
                                               2015.0,
                                               2016.0,
                                               2017.0,
                                               2018.0,
                                               2019.0]

### Input : Feature columns
### Target variable : Neighborhood Rank

In [0]:
X = parking_final[feature_cols] # Features
y = parking_final.RANK

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

### Normalization of the inputs

In [0]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### **Random forest Regressor**

In [0]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

### Sample dataframe of actual and predcited Ranks

In [0]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred,})
df

,Actual,Predicted
952053,1,14.594374
2150132,40,17.838917
1520236,24,16.624840
1568810,24,19.303079
1490843,22,14.526487
...,...,...
2529955,18,16.265856
22932,36,20.038058
2304891,23,20.532279
1951624,27,15.580360


### Accuracy metrics

In [0]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 10.542114967587207
Mean Squared Error: 151.1185791403219
Root Mean Squared Error: 12.293029697365979


In [0]:
parking_final.columns

Index([                              'SUMMONS NUMBER',
                                       'SUMMONS DATE',
                                     'VIOLATION TIME',
                              'VIOLATION DESCRIPTION',
                                      'ORIGINAL FINE',
                           'VIOLATION ADDRESS NUMBER',
                                   'VIOLATION STREET',
                             'VIOLATION FULL ADDRESS',
                                               'CITY',
                                              'STATE',
                                           'LATITUDE',
                                          'LONGITUDE',
                                           'LOCATION',
                                   'COUNCIL DISTRICT',
                                    'POLICE DISTRICT',
                                       'CENSUS TRACT',
                                 'CENSUS BLOCK GROUP',
                                       'CENSUS BLOCK',
          

In [0]:
parking_final['CLASSIFCATION ']

0          25
1          25
2          25
3          25
4          25
           ..
2812642    75
2812643    75
2812644    75
2812645    75
2812646    75
Name: CLASSIFCATION , Length: 2812647, dtype: int64

In [0]:
X1 = parking_final[feature_cols] # Features
y1 = parking_final['CLASS'] 

In [0]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=1)

### Random forest classifier
#### Input : Feature columns
#### Target Variable : Ranking categories (25,50,75,100)

In [0]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X1_train,y1_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:

y1_pred=clf.predict(X1_test)

### Classification Accuracy

In [0]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y1_test, y1_pred))

Accuracy: 0.3773677255731546


### Labelling the Dataset against Assessment Price

In [0]:
assessment_price=pd.read_csv('/content/drive/My Drive/Gentrification/census_rank_Assessmentprice.csv')

In [0]:
assessment_price['CENSUS TRACT'] = pd.to_numeric(assessment_price['CENSUS TRACT'], errors='coerce')
assessment_price = assessment_price.dropna(subset=['CENSUS TRACT'])
assessment_price['CENSUS TRACT']=assessment_price['CENSUS TRACT'].astype(float)

In [0]:
parking_ranked_assessment=parking_summonses.merge(assessment_price,on='CENSUS TRACT')

In [0]:
parking_ranked_assessment.dropna()

,SUMMONS NUMBER,SUMMONS DATE,VIOLATION TIME,VIOLATION DESCRIPTION,ORIGINAL FINE,VIOLATION ADDRESS NUMBER,VIOLATION STREET,VIOLATION FULL ADDRESS,CITY,STATE,LATITUDE,LONGITUDE,LOCATION,COUNCIL DISTRICT,POLICE DISTRICT,CENSUS TRACT,CENSUS BLOCK GROUP,CENSUS BLOCK,Price/sqft,Rank
0,X2308148,08/02/2019,0125P,PARKING VIOLATION,40.0,175.0,FRANKLIN,175 FRANKLIN,Buffalo,NY,42.888195,-78.875835,"(42.8881947942094, -78.87583500193372)",ELLICOTT,District B,165.0,1,1028,272.117490,2
1,X2308149,08/02/2019,0130P,NO STANDING,40.0,37.0,HURON WEST,37 HURON WEST,Buffalo,NY,42.888948,-78.874872,"(42.88894814599496, -78.87487202385579)",ELLICOTT,District B,165.0,1,1022,272.117490,2
2,X2308151,08/02/2019,0305P,METER OVERTIME,40.0,228.0,FRANKLIN,228 FRANKLIN,Buffalo,NY,42.890040,-78.875395,"(42.89003992129964, -78.87539503306608)",ELLICOTT,District B,165.0,1,1023,272.117490,2
3,X2308152,08/02/2019,0312P,METER OVERTIME,40.0,210.0,FRANKLIN,210 FRANKLIN,Buffalo,NY,42.889319,-78.875628,"(42.88931932925486, -78.87562789846032)",ELLICOTT,District B,165.0,1,1023,272.117490,2
4,X2308153,08/02/2019,0320P,METER OVERTIME,40.0,22.0,SENECA WEST,22 SENECA WEST,Buffalo,NY,42.880596,-78.875566,"(42.880595539253534, -78.87556606949188)",FILLMORE,District B,165.0,1,1114,272.117490,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109939,Y9844732,06/11/2015,1226A,EXPIRED INSPECTION STICKER (ALL),35.0,100.0,HOWARD,100 HOWARD,Buffalo,NY,42.883386,-78.850221,"(42.88338630523, -78.850220902355)",ELLICOTT,District C,15.0,2,2023,24.060547,66
2109940,Y9844733,06/11/2015,1229A,NO PARKING,35.0,165.0,HOWARD,165 HOWARD,Buffalo,NY,42.883285,-78.847895,"(42.883285141776, -78.847894893586)",FILLMORE,District C,15.0,1,1008,24.060547,66
2109941,Y9844734,06/11/2015,1230A,EXPIRED INSPECTION STICKER (ALL),35.0,165.0,HOWARD,165 HOWARD,Buffalo,NY,42.883285,-78.847895,"(42.883285141776, -78.847894893586)",FILLMORE,District C,15.0,1,1008,24.060547,66
2109942,Z9835576,05/14/2015,1106A,ALTERNATE PARKING,30.0,66.0,WATSON,66 WATSON,Buffalo,NY,42.881833,-78.849974,"(42.881833438232, -78.849974179483)",ELLICOTT,District C,15.0,1,1010,24.060547,66


In [0]:
parking_ranked_assessment['SUMMONS DATE']=parking_ranked_assessment['SUMMONS DATE'].apply(lambda x:x[6] + x[7] +x[8] +x[9]) 
parking_ranked_assessment['SUMMONS DATE']=parking_ranked_assessment['SUMMONS DATE'].apply(lambda x:float(x))

In [0]:
parking_ranked_assessment=parking_ranked_assessment[parking_ranked_assessment['SUMMONS DATE'] != '2019']

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [0]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb_results = lb.fit_transform(parking_ranked_assessment['VIOLATION DESCRIPTION'])
lb_results_df = pd.DataFrame(lb_results, columns=lb.classes_)

print(lb_results_df.head())

   ALTERNATE PARKING  ...  VEHICLE PARKED ON UNPAVED PORTION OR
0                  0  ...                                     0
1                  0  ...                                     0
2                  0  ...                                     0
3                  0  ...                                     0
4                  0  ...                                     0

[5 rows x 39 columns]


In [0]:
parking_final_assessment = pd.concat([parking_ranked_assessment, lb_results_df], axis=1)

print(parking_final_assessment.head())

  SUMMONS NUMBER  ...  VEHICLE PARKED ON UNPAVED PORTION OR
0       X2308148  ...                                     0
1       X2308149  ...                                     0
2       X2308151  ...                                     0
3       X2308152  ...                                     0
4       X2308153  ...                                     0

[5 rows x 59 columns]


In [0]:
feature_cols =['SUMMONS DATE','ALTERNATE PARKING',
       'BLOCKED DRIVEWAY (ALL)',
       'BUS RT. NO PARKING 1:30 TO 7:00 AM 11/15-4/1', 'DOUBLE PARKING',
       'EXPIRED INSPECTION STICKER (ALL)', 'EXPIRED REGISTRATION STICKER',
       'LESS THAN 20 FT FROM CROSSWALK', 'LOADING ZONE', 'METER OVERTIME',
       'METHOD OF PARKING(ALL)', 'NO INSPECTIONS STICKER (ALL)',
       'NO PARK COMMERICAL TRCK ON RESIDENTIAL', 'NO PARKING',
       'NO PARKING OF HOUSE TRAILERS (ALL)', 'NO REGISTRATION STICKER (ALL)',
       'NO REPAIRS IN STREET EXCEPT EMERGENCY', 'NO STANDING',
       'NO STOPPING (ALL)', 'NO VIOLATION', 'ONE LICENSE PLATE MISSING (ALL)',
       'PARKED BEYOND POSTED TIME LIMIT', 'PARKED IN HANDICAPPED AREA',
       'PARKED LESS THAN 15 FT - FIRE HYDRANT',
       'PARKED MORE THAN 12"FROM CURB', 'PARKED MORE THAN 72 HRS. SAME SPOT',
       'PARKED ON BRIDGE OR IN SUBWAY', 'PARKED ON SIDEWALK (ALL)',
       'PARKED OVER 18 FT FOR MORE THAN ONE',
       'PARKING IN WRONG DIRECTION (ALL)',
       'PARKING ON DIVIDED HWYS PROHIBITED',
       'PARKING PROHIBITED IN CONSTRUCTION SITES', 'PARKING VIOLATION',
       'PARKING WITHOUT CONSENT (ALL)', 'PERSISTEN VIOLATORS AND SCOFFLAWS',
       'SWITCHED PLATES (ALL)', 'UNKNOWN VIOLATION',
       'UNREGISTERED VEH.ON PRIVATE PROPERTY',
       'VEHICLE BLOCKING HANDICAP CURB CUT',
       'VEHICLE PARKED ON UNPAVED PORTION OR']

In [0]:
 parking_final_assessment['Rank'].fillna((parking_final_assessment['Rank'].mean()),inplace=True)

In [0]:
X = parking_final_assessment[feature_cols] # Features
y = parking_final_assessment.Rank

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [0]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [0]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred,})
df

,Actual,Predicted
284683,2,28.166680
1219255,34,30.928944
399622,7,14.971995
1388856,4,12.688517
1582979,13,26.921618
...,...,...
586926,3,26.921618
1757866,14,30.988432
211580,2,27.939208
90944,2,12.688517


### Regression accuracy metrics for data which is labelled against assessment price

In [0]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 16.239564983192004
Mean Squared Error: 411.4642491927575
Root Mean Squared Error: 20.28458156316658


### Gradient Boosting Regressor

In [0]:
regressor = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=3,
    learning_rate=1.0
)

In [0]:
regressor.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
from sklearn.metrics import mean_squared_error
errors = [mean_squared_error(y_test, y_pred) for y_pred in regressor.staged_predict(X_test)]
best_n_estimators = np.argmin(errors)

In [0]:
best_regressor = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=best_n_estimators,
    learning_rate=1.0
)
best_regressor.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=2,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Accuracy metrics for gradient boosting regressor

In [0]:
from sklearn.metrics import mean_absolute_error
y_pred = best_regressor.predict(X_test)
mean_absolute_error(y_test, y_pred)

16.587552122932582

## Adding more features
We are adding the housing court cases dataset with parking violations 

In [0]:
df_hcourt = pd.read_csv('/content/drive/My Drive/Gentrification/Housing_Court_Cases.csv')

In [0]:
df_hcourt.shape

(8797, 22)

In [0]:
df_hcourt['CENSUS TRACT'].isnull().sum()

111

In [0]:
df_hcourt.groupby(['CENSUS TRACT','RESOLUTION'])['CASENO'].count()

CENSUS TRACT  RESOLUTION            
1.1           CLOSE FILE REWRITE         1
              CONDITIONAL DISCHARGE      2
              DEFENDANT FINED            4
              DISMISSED                 16
              REFERRED TO DEMOLITION     1
                                        ..
171.0         CLOSE FILE REWRITE         6
              CONDITIONAL DISCHARGE     11
              DEFENDANT FINED           21
              DISMISSED                 86
              IN COMPLIANCE              5
Name: CASENO, Length: 340, dtype: int64

In [0]:
df_hcourt['RESOLUTION'].value_counts()

DISMISSED                      3012
DEFENDANT FINED                 594
CLOSE FILE REWRITE              419
CONDITIONAL DISCHARGE           386
UNCONDITIONAL DISCHARGE          48
IN COMPLIANCE                    36
REFERRED TO DEMOLITION           12
PROPERTY DEMOLISHED               5
CITY OF BUFFALO OWNED             2
DEFENDANT JAILED PER JUDGE        1
CITY OF BUFFALO FORECLOSURE       1
REFERRED TO DISTRICT              1
Name: RESOLUTION, dtype: int64

In [0]:
df_hcourt_new = pd.get_dummies(df_hcourt,columns=['RESOLUTION'])

In [0]:
df_hcourt_new.drop(df_hcourt_new[df_hcourt_new['CENSUS TRACT'].isnull()].index,inplace=True)

In [0]:
df_hcourt_new.shape

(8686, 33)

In [0]:
df_hcourt_new['LASTACTION'] = df_hcourt_new['LASTACTION'].apply(lambda x: x.split(' A')[0])

In [0]:
df_hcourt_new['LASTACTION'] = pd.to_datetime(df_hcourt_new['LASTACTION'])

In [0]:
df_hcourt_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8686 entries, 0 to 8796
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   CASEKEY                                 8686 non-null   int64         
 1   ADDDTTM                                 8686 non-null   object        
 2   CASENO                                  8686 non-null   object        
 3   CASETYPE                                8686 non-null   object        
 4   STATUS                                  8686 non-null   object        
 5   LASTACTION                              8686 non-null   datetime64[ns]
 6   RESDTTM                                 3753 non-null   object        
 7   ADDRESS                                 8686 non-null   object        
 8   CITY                                    8686 non-null   object        
 9   STATE                                   8686 non-nul

In [0]:
df_hcourt_new = df_hcourt_new[df_hcourt_new['LASTACTION'].dt.year == 2018]

In [0]:
df_hcourt_new.iloc[:,21:]=df_hcourt_new.iloc[:,21:].replace(0,np.NaN)

In [0]:
df_hcourt_new.iloc[:,21:]

,RESOLUTION_CITY OF BUFFALO FORECLOSURE,RESOLUTION_CITY OF BUFFALO OWNED,RESOLUTION_CLOSE FILE REWRITE,RESOLUTION_CONDITIONAL DISCHARGE,RESOLUTION_DEFENDANT FINED,RESOLUTION_DEFENDANT JAILED PER JUDGE,RESOLUTION_DISMISSED,RESOLUTION_IN COMPLIANCE,RESOLUTION_PROPERTY DEMOLISHED,RESOLUTION_REFERRED TO DEMOLITION,RESOLUTION_REFERRED TO DISTRICT,RESOLUTION_UNCONDITIONAL DISCHARGE
14,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_count_hcourt = df_hcourt_new.groupby('CENSUS TRACT').count()

In [0]:
df_count_hcourt['CENSUS TRACT']=df_count_hcourt.index

In [0]:
df_count_hcourt

,CASEKEY,ADDDTTM,CASENO,CASETYPE,STATUS,LASTACTION,RESDTTM,ADDRESS,CITY,STATE,ZIP,CONTACT,LOCATION,LATITUDE,LONGITUDE,COUNCIL DISTRICT,POLICE DISTRICT,CENSUS BLOCK GROUP,CENSUS BLOCK,NEIGHBORHOOD,RESOLUTION_CITY OF BUFFALO FORECLOSURE,RESOLUTION_CITY OF BUFFALO OWNED,RESOLUTION_CLOSE FILE REWRITE,RESOLUTION_CONDITIONAL DISCHARGE,RESOLUTION_DEFENDANT FINED,RESOLUTION_DEFENDANT JAILED PER JUDGE,RESOLUTION_DISMISSED,RESOLUTION_IN COMPLIANCE,RESOLUTION_PROPERTY DEMOLISHED,RESOLUTION_REFERRED TO DEMOLITION,RESOLUTION_REFERRED TO DISTRICT,RESOLUTION_UNCONDITIONAL DISCHARGE,CENSUS TRACT
CENSUS TRACT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.1,6,6,6,6,6,6,1,6,6,6,6,6,6,6,6,6,6,6,6,6,0,0,0,0,0,0,1,0,0,0,0,0,1.1
2.0,6,6,6,6,6,6,0,6,6,6,6,6,6,6,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,2.0
5.0,14,14,14,14,14,14,5,14,14,14,14,14,14,14,14,14,14,14,14,14,0,0,0,0,0,0,5,0,0,0,0,0,5.0
6.0,4,4,4,4,4,4,1,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0,0,1,0,0,0,0,0,6.0
7.0,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167.0,4,4,4,4,4,4,2,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0,0,2,0,0,0,0,0,167.0
168.0,33,33,33,33,33,33,6,33,33,33,33,33,33,33,33,33,33,33,33,33,0,0,0,0,0,0,7,0,0,0,0,0,168.0
169.0,3,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,1,0,0,0,0,0,169.0


In [0]:
df_count_hcourt.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 72 entries, 1.1 to 171.0
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   CASEKEY                                 72 non-null     int64  
 1   ADDDTTM                                 72 non-null     int64  
 2   CASENO                                  72 non-null     int64  
 3   CASETYPE                                72 non-null     int64  
 4   STATUS                                  72 non-null     int64  
 5   LASTACTION                              72 non-null     int64  
 6   RESDTTM                                 72 non-null     int64  
 7   ADDRESS                                 72 non-null     int64  
 8   CITY                                    72 non-null     int64  
 9   STATE                                   72 non-null     int64  
 10  ZIP                                     72 non-null     i

In [0]:
df_courtcases =df_count_hcourt[['RESOLUTION_IN COMPLIANCE','RESOLUTION_PROPERTY DEMOLISHED','RESOLUTION_REFERRED TO DEMOLITION','RESOLUTION_REFERRED TO DISTRICT','RESOLUTION_UNCONDITIONAL DISCHARGE','RESOLUTION_DISMISSED','RESOLUTION_DEFENDANT JAILED PER JUDGE','RESOLUTION_DEFENDANT FINED','RESOLUTION_CONDITIONAL DISCHARGE','RESOLUTION_CITY OF BUFFALO FORECLOSURE','RESOLUTION_CITY OF BUFFALO OWNED','RESOLUTION_CLOSE FILE REWRITE']]

In [0]:
df_courtcases.reset_index(level=0,inplace=True)

In [0]:
df_courtcases.head()

,CENSUS TRACT,RESOLUTION_IN COMPLIANCE,RESOLUTION_PROPERTY DEMOLISHED,RESOLUTION_REFERRED TO DEMOLITION,RESOLUTION_REFERRED TO DISTRICT,RESOLUTION_UNCONDITIONAL DISCHARGE,RESOLUTION_DISMISSED,RESOLUTION_DEFENDANT JAILED PER JUDGE,RESOLUTION_DEFENDANT FINED,RESOLUTION_CONDITIONAL DISCHARGE,RESOLUTION_CITY OF BUFFALO FORECLOSURE,RESOLUTION_CITY OF BUFFALO OWNED,RESOLUTION_CLOSE FILE REWRITE
0,1.1,0,0,0,0,0,1,0,0,0,0,0,0
1,2.0,0,0,0,0,0,0,0,0,0,0,0,0
2,5.0,0,0,0,0,0,5,0,0,0,0,0,0
3,6.0,0,0,0,0,0,1,0,0,0,0,0,0
4,7.0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_parking = pd.read_csv('/content/drive/My Drive/Gentrification/Parking_Summonses .csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_parking.drop(df_parking[df_parking['CENSUS TRACT']=='UNKNOWN'].index,inplace=True)

In [0]:
df_parking.shape

(2153798, 18)

In [0]:
df_parking['CENSUS TRACT'] = pd.to_numeric(df_parking['CENSUS TRACT'],errors='coerce')

In [0]:
df_parking['CENSUS TRACT'].unique()

array([   nan, 165.  ,  68.  ,  71.02,  66.01,  66.02,  65.01,  23.  ,
         9.  ,  10.  ,  63.01,  28.  , 171.  ,  58.02,  31.  ,  63.02,
        69.02,  55.  ,  67.01,  52.02,  71.01,  47.  ,  61.  , 164.  ,
       169.  ,  70.  ,  69.01,  67.02,  24.  ,  30.  ,  37.  ,   8.  ,
       167.  ,  52.01,  59.  ,  58.01,  57.  ,   5.  ,  17.  ,  43.  ,
        25.02,  29.  ,  49.  ,  50.  ,  48.  ,  45.  ,  72.02,  54.  ,
        33.01,  11.  ,  51.  ,  33.02,  35.  ,  38.  ,   7.  ,  34.  ,
        14.02, 170.  ,  53.  ,  16.  , 166.  ,   1.1 ,   6.  ,   2.  ,
       163.  ,  19.  , 168.  ,  27.02,  41.  ,  46.01,  42.  ,  56.  ,
        40.01,  46.02,  44.01,  39.01,  44.02,  36.  ,  15.  ,  62.01])

In [0]:
df_parking.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2153798 entries, 0 to 2155858
Data columns (total 18 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   SUMMONS NUMBER            object 
 1   SUMMONS DATE              object 
 2   VIOLATION TIME            object 
 3   VIOLATION DESCRIPTION     object 
 4   ORIGINAL FINE             float64
 5   VIOLATION ADDRESS NUMBER  float64
 6   VIOLATION STREET          object 
 7   VIOLATION FULL ADDRESS    object 
 8   CITY                      object 
 9   STATE                     object 
 10  LATITUDE                  float64
 11  LONGITUDE                 float64
 12  LOCATION                  object 
 13  COUNCIL DISTRICT          object 
 14  POLICE DISTRICT           object 
 15  CENSUS TRACT              float64
 16  CENSUS BLOCK GROUP        object 
 17  CENSUS BLOCK              object 
dtypes: float64(5), object(13)
memory usage: 312.2+ MB


In [0]:
df_parking['SUMMONS DATE']=pd.to_datetime(df_parking['SUMMONS DATE'])

In [0]:
df_parking = df_parking[df_parking['SUMMONS DATE'].dt.year == 2018]

In [0]:
df_rank=pd.read_csv('/content/drive/My Drive/Gentrification/census_rank_Assessmentprice.csv')

In [0]:
df_merged_parking = df_parking.merge(df_rank,left_on = 'CENSUS TRACT',right_on='CENSUS TRACT',how='inner')

In [0]:
df_merged_parking = df_merged_parking.merge(df_courtcases,left_on = 'CENSUS TRACT',right_on='CENSUS TRACT',how='inner')

In [0]:
df_merged_parking.shape




(178518, 32)

In [0]:
df_merged_parking['Rank'].isnull().sum()

0

In [0]:
df_merged_parking.drop(df_merged_parking[df_merged_parking['Rank'].isnull()].index,inplace=True)

In [0]:
df_parking.drop(df_parking[df_parking['CENSUS TRACT'].isnull()].index,inplace=True)

In [0]:
df_merged_parking['Rank'].isnull().sum()

0

In [0]:
df_merged_parking.to_csv('Parking_ranked.csv',index=False)

In [0]:
df_merged_parking.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178518 entries, 0 to 178517
Data columns (total 32 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   SUMMONS NUMBER                          178518 non-null  object        
 1   SUMMONS DATE                            178518 non-null  datetime64[ns]
 2   VIOLATION TIME                          178512 non-null  object        
 3   VIOLATION DESCRIPTION                   178518 non-null  object        
 4   ORIGINAL FINE                           178518 non-null  float64       
 5   VIOLATION ADDRESS NUMBER                178513 non-null  float64       
 6   VIOLATION STREET                        178513 non-null  object        
 7   VIOLATION FULL ADDRESS                  178518 non-null  object        
 8   CITY                                    178518 non-null  object        
 9   STATE                                

In [0]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
df_merged_parking_temp = lb.fit_transform(df_merged_parking['VIOLATION DESCRIPTION'])
df_merged_parking_temp2 = pd.DataFrame(df_merged_parking_temp, columns=lb.classes_)

print(df_merged_parking.head())

  SUMMONS NUMBER  ... RESOLUTION_CLOSE FILE REWRITE
0       32055761  ...                             0
1       H2205740  ...                             0
2       72095850  ...                             0
3       A9645156  ...                             0
4       A9781763  ...                             0

[5 rows x 32 columns]


In [0]:
df_merged_parking

,SUMMONS NUMBER,SUMMONS DATE,VIOLATION TIME,VIOLATION DESCRIPTION,ORIGINAL FINE,VIOLATION ADDRESS NUMBER,VIOLATION STREET,VIOLATION FULL ADDRESS,CITY,STATE,LATITUDE,LONGITUDE,LOCATION,COUNCIL DISTRICT,POLICE DISTRICT,CENSUS TRACT,CENSUS BLOCK GROUP,CENSUS BLOCK,Price/sqft,Rank,RESOLUTION_IN COMPLIANCE,RESOLUTION_PROPERTY DEMOLISHED,RESOLUTION_REFERRED TO DEMOLITION,RESOLUTION_REFERRED TO DISTRICT,RESOLUTION_UNCONDITIONAL DISCHARGE,RESOLUTION_DISMISSED,RESOLUTION_DEFENDANT JAILED PER JUDGE,RESOLUTION_DEFENDANT FINED,RESOLUTION_CONDITIONAL DISCHARGE,RESOLUTION_CITY OF BUFFALO FORECLOSURE,RESOLUTION_CITY OF BUFFALO OWNED,RESOLUTION_CLOSE FILE REWRITE
0,32055761,2018-02-16,0659A,ALTERNATE PARKING,40.0,7.0,ORTON,7 ORTON,Buffalo,NY,42.899591,-78.882205,"(42.899591383921, -78.882205118133)",ELLICOTT,District B,68.0,3,3003,155.870927,7,0,0,0,0,0,0,0,0,0,0,0,0
1,H2205740,2018-12-18,1155A,NO PARKING,40.0,960.0,MAIN,960 MAIN,Buffalo,NY,42.899943,-78.869785,"(42.899942641686, -78.869785312295)",FILLMORE,District B,68.0,2,2000,155.870927,7,0,0,0,0,0,0,0,0,0,0,0,0
2,72095850,2018-05-07,0435P,METER OVERTIME,40.0,498.0,ELMWOOD,498 ELMWOOD,Buffalo,NY,42.896816,-78.877154,"(42.896816346134, -78.877154188574)",FILLMORE,District B,68.0,3,3009,155.870927,7,0,0,0,0,0,0,0,0,0,0,0,0
3,A9645156,2018-07-26,0247P,NO STANDING,40.0,158.0,PEARL,158 PEARL,Buffalo,NY,42.900219,-78.870895,"(42.90021880526, -78.870894823926)",FILLMORE,District B,68.0,2,2001,155.870927,7,0,0,0,0,0,0,0,0,0,0,0,0
4,A9781763,2018-10-13,0130A,NO PARKING,40.0,221.0,ALLEN,221 ALLEN,Buffalo,NY,42.899519,-78.878243,"(42.899518778881, -78.878242864676)",FILLMORE,District B,68.0,1,1002,155.870927,7,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178513,X2204389,2018-12-18,0948A,ALTERNATE PARKING,40.0,268.0,KRETTNER,268 KRETTNER,Buffalo,NY,42.890394,-78.847980,"(42.890394064688, -78.847979738294)",ELLICOTT,District C,15.0,2,2004,24.060547,66,0,0,0,0,0,7,0,0,0,0,0,0
178514,X2208482,2018-12-26,0958A,ALTERNATE PARKING,40.0,268.0,SHERMAN,268 SHERMAN,Buffalo,NY,42.890968,-78.847000,"(42.890967640127, -78.846999593045)",ELLICOTT,District C,15.0,2,2003,24.060547,66,0,0,0,0,0,7,0,0,0,0,0,0
178515,X2208889,2018-12-27,0949A,ALTERNATE PARKING,40.0,351.0,ADAMS,351 ADAMS,Buffalo,NY,42.889734,-78.850854,"(42.889733600078, -78.850853884353)",ELLICOTT,District C,15.0,2,2006,24.060547,66,0,0,0,0,0,7,0,0,0,0,0,0
178516,X2208890,2018-12-27,1000A,ALTERNATE PARKING,40.0,245.0,SHUMWAY,245 SHUMWAY,Buffalo,NY,42.890234,-78.844912,"(42.890234013335, -78.844912024511)",FILLMORE,District C,15.0,2,2000,24.060547,66,0,0,0,0,0,7,0,0,0,0,0,0


In [0]:
df_merged_parking = pd.concat([df_merged_parking, df_merged_parking_temp2], axis=1)

print(df_merged_parking.head())

  SUMMONS NUMBER  ... VEHICLE PARKED ON UNPAVED PORTION OR
0       32055761  ...                                    0
1       H2205740  ...                                    0
2       72095850  ...                                    0
3       A9645156  ...                                    0
4       A9781763  ...                                    0

[5 rows x 69 columns]


In [0]:
for col in df_merged_parking_final:
  print("'" + col + "'" + ',')

'SUMMONS NUMBER',
'SUMMONS DATE',
'VIOLATION TIME',
'VIOLATION DESCRIPTION',
'ORIGINAL FINE',
'VIOLATION ADDRESS NUMBER',
'VIOLATION STREET',
'VIOLATION FULL ADDRESS',
'CITY',
'STATE',
'LATITUDE',
'LONGITUDE',
'LOCATION',
'COUNCIL DISTRICT',
'POLICE DISTRICT',
'CENSUS TRACT',
'CENSUS BLOCK GROUP',
'CENSUS BLOCK',
'Price/sqft',
'Rank',
'RESOLUTION_IN COMPLIANCE',
'RESOLUTION_PROPERTY DEMOLISHED',
'RESOLUTION_REFERRED TO DEMOLITION',
'RESOLUTION_REFERRED TO DISTRICT',
'RESOLUTION_UNCONDITIONAL DISCHARGE',
'RESOLUTION_DISMISSED',
'RESOLUTION_DEFENDANT JAILED PER JUDGE',
'RESOLUTION_DEFENDANT FINED',
'RESOLUTION_CONDITIONAL DISCHARGE',
'RESOLUTION_CITY OF BUFFALO FORECLOSURE',
'RESOLUTION_CITY OF BUFFALO OWNED',
'RESOLUTION_CLOSE FILE REWRITE',
'ALTERNATE PARKING',
'BLOCKED DRIVEWAY (ALL)',
'BUS RT. NO PARKING 1:30 TO 7:00 AM 11/15-4/1',
'DOUBLE PARKING',
'EXPIRED INSPECTION STICKER (ALL)',
'EXPIRED REGISTRATION STICKER',
'LESS THAN 20 FT FROM CROSSWALK',
'LOADING ZONE',
'METER OVERTIME',


In [0]:
X=df_merged_parking[['ALTERNATE PARKING', 'BLOCKED DRIVEWAY (ALL)',
       'BUS RT. NO PARKING 1:30 TO 7:00 AM 11/15-4/1', 'DOUBLE PARKING',
       'EXPIRED INSPECTION STICKER (ALL)', 'EXPIRED REGISTRATION STICKER',
       'LESS THAN 20 FT FROM CROSSWALK', 'LOADING ZONE', 'METER OVERTIME',
       'METHOD OF PARKING(ALL)', 'NO INSPECTIONS STICKER (ALL)',
       'NO PARK COMMERICAL TRCK ON RESIDENTIAL', 'NO PARKING',
       'NO PARKING OF HOUSE TRAILERS (ALL)', 'NO REGISTRATION STICKER (ALL)',
       'NO REPAIRS IN STREET EXCEPT EMERGENCY', 'NO STANDING',
       'NO STOPPING (ALL)', 'NO VIOLATION', 'ONE LICENSE PLATE MISSING (ALL)',
       'PARKED BEYOND POSTED TIME LIMIT', 'PARKED IN HANDICAPPED AREA',
       'PARKED LESS THAN 15 FT - FIRE HYDRANT',
       'PARKED MORE THAN 12"FROM CURB', 'PARKED MORE THAN 72 HRS. SAME SPOT',
       'PARKED ON SIDEWALK (ALL)', 'PARKED OVER 18 FT FOR MORE THAN ONE',
       'PARKING IN WRONG DIRECTION (ALL)',
       'PARKING ON DIVIDED HWYS PROHIBITED',
       'PARKING PROHIBITED IN CONSTRUCTION SITES', 'PARKING VIOLATION',
       'PARKING WITHOUT CONSENT (ALL)', 'PERSISTEN VIOLATORS AND SCOFFLAWS',
       'SWITCHED PLATES (ALL)', 'UNKNOWN VIOLATION',
       'VEHICLE BLOCKING HANDICAP CURB CUT',
       'VEHICLE PARKED ON UNPAVED PORTION OR','RESOLUTION_IN COMPLIANCE','RESOLUTION_PROPERTY DEMOLISHED','RESOLUTION_REFERRED TO DEMOLITION','RESOLUTION_REFERRED TO DISTRICT','RESOLUTION_UNCONDITIONAL DISCHARGE','RESOLUTION_DEFENDANT JAILED PER JUDGE','RESOLUTION_DEFENDANT FINED','RESOLUTION_CONDITIONAL DISCHARGE','RESOLUTION_CITY OF BUFFALO FORECLOSURE','RESOLUTION_CITY OF BUFFALO OWNED','RESOLUTION_CLOSE FILE REWRITE']]

In [0]:
y=df_merged_parking['Rank']


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
rfc = RandomForestRegressor(n_estimators=600)

In [0]:
rfc.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=600, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
y_pred = rfc.predict(X_test)

In [0]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 14.404602047615668
Mean Squared Error: 327.02138588710824
Root Mean Squared Error: 18.083732631487013


In [0]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df.head(10)

,Actual,Predicted
70852,20,30.848281
99070,21,37.053758
105853,50,30.848281
84656,5,30.848281
111174,44,30.848281
67118,20,30.848281
72823,20,30.848281
132390,34,30.848281
140425,14,37.053758
8378,7,15.960545


In [0]:
pd.DataFrame(rfc.feature_importances_, X_train.columns).sort_values(by=0).tail(10)

,0
METHOD OF PARKING(ALL),0.004560
PARKED ON SIDEWALK (ALL),0.007973
VEHICLE PARKED ON UNPAVED PORTION OR,0.008156
EXPIRED INSPECTION STICKER (ALL),0.009448
NO PARKING,0.019527
NO STANDING,0.020077
RESOLUTION_CONDITIONAL DISCHARGE,0.047683
METER OVERTIME,0.069115
ALTERNATE PARKING,0.240371
RESOLUTION_DEFENDANT FINED,0.548468


In [0]:
df_codeviolations=pd.read_csv('/content/drive/My Drive/Gentrification/Code_Violations.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_codeviolations['CENSUS TRACT'].isnull().sum()

1045

In [0]:
df_codeviolations.groupby(['CENSUS TRACT','CODE SECTION'])['CASE NUMBER'].count()

CENSUS TRACT  CODE SECTION                    
1.1           Automatic Fire Sprinkler Systems     1
              Buffalo Ordinace                     3
              Buffalo Ordinance                   55
              Carbon Monoxide Detection            5
              Component Serviceability             2
                                                  ..
9             Property Maintenance                45
              Residential                          3
              Rubbish and Garbage                 10
              V7MIGRATION                          1
UNKNOWN       Buffalo Ordinance                    1
Name: CASE NUMBER, Length: 4305, dtype: int64

In [0]:
df_codeviolations['CODE SECTION'].value_counts()

Exterior Structure                 27582
Property Maintenance               13828
Buffalo Ordinance                  11655
Exterior Property Areas            10023
Rubbish and Garbage                 6519
                                   ...  
Temporary Uses                         1
Unsafe Structures and Equioment        1
Accessibility                          1
Toilet, Bath and Shower Spaces         1
Liquefied petroleum gases              1
Name: CODE SECTION, Length: 132, dtype: int64

In [0]:
df_codeviolations_new = pd.get_dummies(df_codeviolations,columns=['CODE SECTION'])

In [0]:
df_codeviolations_new.drop(df_codeviolations_new[df_codeviolations_new['CENSUS TRACT'].isnull()].index,inplace=True)

In [0]:
df_codeviolations_new['DATE'] = df_codeviolations_new['DATE'].apply(lambda x: x.split(' A')[0])

In [0]:
df_codeviolations_new['DATE'] = pd.to_datetime(df_codeviolations_new['DATE'])

In [0]:
df_codeviolations_new = df_codeviolations_new[df_codeviolations_new['DATE'].dt.year == 2018]


In [0]:

#df_codeviolations_new.iloc[:,23:]=df_codeviolations_new.iloc[:,23:].replace(0,np.NaN)

In [0]:
#df_codeviolations_new.iloc[:,23:]

In [0]:
df_count_codeviolations = df_codeviolations_new.groupby('CENSUS TRACT').count()

In [0]:
df_count_codeviolations['CENSUS TRACT']=df_count_codeviolations.index

In [0]:
for col in df_count_codeviolations:
  print("'" + col + "'" + ',')

'UNIQUEKEY',
'DATE',
'CASE NUMBER',
'CASE TYPE',
'STATUS',
'PROPCLASS',
'ADDRESS',
'CITY',
'STATE',
'ZIP',
'CODE',
'DESCRIPTION',
'COMMENTS',
'VIOLATION LOCATION',
'INSPECTOR',
'LOCATION',
'LATITUDE',
'LONGITUDE',
'COUNCIL DISTRICT',
'POLICE DISTRICT',
'CENSUS BLOCK GROUP',
'CENSUS BLOCK',
'NEIGHBORHOOD',
'CODE SECTION_Access by occupant',
'CODE SECTION_Access to Building Openings and Roofs',
'CODE SECTION_Accessibility',
'CODE SECTION_Accessory Uses',
'CODE SECTION_Administration',
'CODE SECTION_Applicability',
'CODE SECTION_Art.3 Neighborhood Zones',
'CODE SECTION_Art.6 Uses',
'CODE SECTION_Art.7 Site Development',
'CODE SECTION_Artical 6 Uses',
'CODE SECTION_Artical 9 Signs',
'CODE SECTION_Article 7 Site Development',
'CODE SECTION_Article 7 site development',
'CODE SECTION_Article 9 Signs',
'CODE SECTION_Automatic Fire Sprinkler Systems',
'CODE SECTION_BUFFALO',
'CODE SECTION_Buffalo Ordinace',
'CODE SECTION_Buffalo Ordinance',
'CODE SECTION_Build Services & Systems',
'CODE SECTION

In [0]:
df_codeviolations =df_count_codeviolations[['CODE SECTION_Access by occupant',
'CODE SECTION_Access to Building Openings and Roofs',
'CODE SECTION_Accessibility',
'CODE SECTION_Accessory Uses',
'CODE SECTION_Administration',
'CODE SECTION_Applicability',
'CODE SECTION_Art.3 Neighborhood Zones',
'CODE SECTION_Art.6 Uses',
'CODE SECTION_Art.7 Site Development',
'CODE SECTION_Artical 6 Uses',
'CODE SECTION_Artical 9 Signs',
'CODE SECTION_Article 7 Site Development',
'CODE SECTION_Article 7 site development',
'CODE SECTION_Article 9 Signs',
'CODE SECTION_Automatic Fire Sprinkler Systems',
'CODE SECTION_BUFFALO',
'CODE SECTION_Buffalo Ordinace',
'CODE SECTION_Buffalo Ordinance',
'CODE SECTION_Build Services & Systems',
'CODE SECTION_Building Planning',
'CODE SECTION_Buisness License',
'CODE SECTION_Business License',
'CODE SECTION_Carbon Monoxide Detection',
'CODE SECTION_Carbon monoxide detection',
'CODE SECTION_Certificate of occupancy',
'CODE SECTION_Chimneys and Vents',
'CODE SECTION_Combustible wast material',
'CODE SECTION_Commercial & Recreational Vehicles',
'CODE SECTION_Component Serviceability',
'CODE SECTION_Curb Cuts and Driveways',
'CODE SECTION_Design Criteria',
'CODE SECTION_Detached House',
'CODE SECTION_Drain Pipe Materials and Sizes',
'CODE SECTION_Duct Systems',
'CODE SECTION_Duties and Powers of the Code Official',
'CODE SECTION_EQUIP / APPLIANCE LOCATION',
'CODE SECTION_EXISTING BUILDING CODE',
'CODE SECTION_EXISTING R OCCUPANCIES',
'CODE SECTION_Electical Facilities',
'CODE SECTION_Electrical Equipment',
'CODE SECTION_Electrical Facilities',
'CODE SECTION_Elevators, Escalators and Dumbwaiters',
'CODE SECTION_Emergency Measures',
'CODE SECTION_Emergency power for illumination',
'CODE SECTION_Existing non-required equipment and systems',
'CODE SECTION_Exterior Property Areas',
'CODE SECTION_Exterior Structure',
'CODE SECTION_FIRE CODE',
'CODE SECTION_Fences and Walls',
'CODE SECTION_Fire Code',
'CODE SECTION_Fire Protection Systems',
'CODE SECTION_Fire Protection and Safety',
'CODE SECTION_Fire and Smoke Protection',
'CODE SECTION_Fire-Resistance Ratings',
'CODE SECTION_Fire-Resistant Construction',
'CODE SECTION_Foundations',
'CODE SECTION_GENERAL FIRE SAFETY REQ.',
'CODE SECTION_Gen. Regs. (Condensate)',
'CODE SECTION_Gen. Regs. (Equip/Appli. locations)',
'CODE SECTION_Gen. Regs. (Repair)',
'CODE SECTION_General',
'CODE SECTION_General (Interior)',
'CODE SECTION_General Regulations',
'CODE SECTION_General Requirements',
'CODE SECTION_General requirements',
'CODE SECTION_Glazing',
'CODE SECTION_Guards and Window Fall Protection',
'CODE SECTION_Handrails',
'CODE SECTION_Handrails and Guardrails',
'CODE SECTION_Heating Facilities',
'CODE SECTION_Interior Structure',
'CODE SECTION_LANDSCAPE',
'CODE SECTION_Light',
'CODE SECTION_Light, Ventilation and Heating',
'CODE SECTION_Liquefied Petroleum Gases',
'CODE SECTION_Liquefied petroleum gases',
'CODE SECTION_Means of Egress',
'CODE SECTION_Means of Egress Illumination',
'CODE SECTION_Means of egress',
'CODE SECTION_Mechanical Equipment',
'CODE SECTION_Occupancy Limitations',
'CODE SECTION_Open Burning',
'CODE SECTION_Open Flames',
'CODE SECTION_PLUMBING CODE',
'CODE SECTION_Parking Lots',
'CODE SECTION_Pedestrain Walkways and tunnels',
'CODE SECTION_Pest Elimination',
'CODE SECTION_Plumbing Facilities and Fixture Requirements',
'CODE SECTION_Plumbing Systems and Fixtures',
'CODE SECTION_Projection of Pedestrians',
'CODE SECTION_Property Maint Supplement',
'CODE SECTION_Property Maintenance',
'CODE SECTION_Real Property Actions & Proceedings',
'CODE SECTION_Required Facilities',
'CODE SECTION_Residential',
'CODE SECTION_Roof Assemblies',
'CODE SECTION_Roof recover',
'CODE SECTION_Rubbish and Garbage',
'CODE SECTION_Safeguards during const.',
'CODE SECTION_Sanitary Drainage System',
'CODE SECTION_Sanitation',
'CODE SECTION_Scope and Application',
'CODE SECTION_Shop-Front House',
'CODE SECTION_Site Development',
'CODE SECTION_Site Devlopment',
'CODE SECTION_Site Impacts',
'CODE SECTION_Site Landscape',
'CODE SECTION_Smoke Partitions',
'CODE SECTION_Stairways',
'CODE SECTION_Storm Drainage',
'CODE SECTION_Stpo Work Order',
'CODE SECTION_Structural Design',
'CODE SECTION_Supplement',
'CODE SECTION_Supplement 2017',
'CODE SECTION_Swimming Pools, Spas and Hot Tubs',
'CODE SECTION_Swimming pools spas and hot tubs',
'CODE SECTION_Swimming pools, spas and hot tubs',
'CODE SECTION_Temporary Uses',
'CODE SECTION_The Building Code',
'CODE SECTION_Time and access',
'CODE SECTION_Toilet Rooms',
'CODE SECTION_Toilet, Bath and Shower Spaces',
'CODE SECTION_Unsafe Structures and Equioment',
'CODE SECTION_Unsafe Structures and Equipment',
'CODE SECTION_Unsafe structures and equipment',
'CODE SECTION_Unvented room heaters',
'CODE SECTION_Uses',
'CODE SECTION_V7MIGRATION',
'CODE SECTION_Ventilation',
'CODE SECTION_Venting of appliances',
'CODE SECTION_Water Heaters',
'CODE SECTION_Water Systems',
]]

In [0]:
df_codeviolations.reset_index(level=0,inplace=True)

In [0]:
df_merged_parking_code = df_parking.merge(df_rank,left_on = 'CENSUS TRACT',right_on='CENSUS TRACT',how='inner')

In [0]:
df_codeviolations['CENSUS TRACT']=df_codeviolations['CENSUS TRACT'].astype(str).astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_merged_parking_code['CENSUS TRACT'].dtype

dtype('float64')

In [0]:
df_codeviolations['CENSUS TRACT'].dtype

dtype('float64')

In [0]:
df_merged_parking_code = df_merged_parking.merge(df_codeviolations,left_on = 'CENSUS TRACT',right_on='CENSUS TRACT',how='inner')

In [0]:
df_merged_parking_code.columns

Index(['SUMMONS NUMBER', 'SUMMONS DATE', 'VIOLATION TIME',
       'VIOLATION DESCRIPTION', 'ORIGINAL FINE', 'VIOLATION ADDRESS NUMBER',
       'VIOLATION STREET', 'VIOLATION FULL ADDRESS', 'CITY', 'STATE',
       ...
       'CODE SECTION_Unsafe Structures and Equioment',
       'CODE SECTION_Unsafe Structures and Equipment',
       'CODE SECTION_Unsafe structures and equipment',
       'CODE SECTION_Unvented room heaters', 'CODE SECTION_Uses',
       'CODE SECTION_V7MIGRATION', 'CODE SECTION_Ventilation',
       'CODE SECTION_Venting of appliances', 'CODE SECTION_Water Heaters',
       'CODE SECTION_Water Systems'],
      dtype='object', length=201)

In [0]:
len(df_merged_parking_code)

355348

In [0]:
for col in df_merged_parking_code:
  print("'" + col + "'" + ',')

'SUMMONS NUMBER',
'SUMMONS DATE',
'VIOLATION TIME',
'VIOLATION DESCRIPTION',
'ORIGINAL FINE',
'VIOLATION ADDRESS NUMBER',
'VIOLATION STREET',
'VIOLATION FULL ADDRESS',
'CITY',
'STATE',
'LATITUDE',
'LONGITUDE',
'LOCATION',
'COUNCIL DISTRICT',
'POLICE DISTRICT',
'CENSUS TRACT',
'CENSUS BLOCK GROUP',
'CENSUS BLOCK',
'Price/sqft',
'Rank',
'RESOLUTION_IN COMPLIANCE',
'RESOLUTION_PROPERTY DEMOLISHED',
'RESOLUTION_REFERRED TO DEMOLITION',
'RESOLUTION_REFERRED TO DISTRICT',
'RESOLUTION_UNCONDITIONAL DISCHARGE',
'RESOLUTION_DISMISSED',
'RESOLUTION_DEFENDANT JAILED PER JUDGE',
'RESOLUTION_DEFENDANT FINED',
'RESOLUTION_CONDITIONAL DISCHARGE',
'RESOLUTION_CITY OF BUFFALO FORECLOSURE',
'RESOLUTION_CITY OF BUFFALO OWNED',
'RESOLUTION_CLOSE FILE REWRITE',
'ALTERNATE PARKING',
'BLOCKED DRIVEWAY (ALL)',
'BUS RT. NO PARKING 1:30 TO 7:00 AM 11/15-4/1',
'DOUBLE PARKING',
'EXPIRED INSPECTION STICKER (ALL)',
'EXPIRED REGISTRATION STICKER',
'LESS THAN 20 FT FROM CROSSWALK',
'LOADING ZONE',
'METER OVERTIME',


In [0]:
X1=df_merged_parking_code[['RESOLUTION_IN COMPLIANCE',
'RESOLUTION_PROPERTY DEMOLISHED',
'RESOLUTION_REFERRED TO DEMOLITION',
'RESOLUTION_REFERRED TO DISTRICT',
'RESOLUTION_UNCONDITIONAL DISCHARGE',

'RESOLUTION_DEFENDANT JAILED PER JUDGE',
'RESOLUTION_DEFENDANT FINED',
'RESOLUTION_CONDITIONAL DISCHARGE',
'RESOLUTION_CITY OF BUFFALO FORECLOSURE',
'RESOLUTION_CITY OF BUFFALO OWNED',
'RESOLUTION_CLOSE FILE REWRITE',
'ALTERNATE PARKING',
'BLOCKED DRIVEWAY (ALL)',
'BUS RT. NO PARKING 1:30 TO 7:00 AM 11/15-4/1',
'DOUBLE PARKING',
'EXPIRED INSPECTION STICKER (ALL)',
'EXPIRED REGISTRATION STICKER',
'LESS THAN 20 FT FROM CROSSWALK',
'LOADING ZONE',
'METER OVERTIME',
'METHOD OF PARKING(ALL)',
'NO INSPECTIONS STICKER (ALL)',
'NO PARK COMMERICAL TRCK ON RESIDENTIAL',
'NO PARKING',
'NO PARKING OF HOUSE TRAILERS (ALL)',
'NO REGISTRATION STICKER (ALL)',
'NO REPAIRS IN STREET EXCEPT EMERGENCY',
'NO STANDING',
'NO STOPPING (ALL)',
'NO VIOLATION',
'ONE LICENSE PLATE MISSING (ALL)',
'PARKED BEYOND POSTED TIME LIMIT',
'PARKED IN HANDICAPPED AREA',
'PARKED LESS THAN 15 FT - FIRE HYDRANT',
'PARKED MORE THAN 12"FROM CURB',
'PARKED MORE THAN 72 HRS. SAME SPOT',
'PARKED ON SIDEWALK (ALL)',
'PARKED OVER 18 FT FOR MORE THAN ONE',
'PARKING IN WRONG DIRECTION (ALL)',
'PARKING ON DIVIDED HWYS PROHIBITED',
'PARKING PROHIBITED IN CONSTRUCTION SITES',
'PARKING VIOLATION',
'PARKING WITHOUT CONSENT (ALL)',
'PERSISTEN VIOLATORS AND SCOFFLAWS',
'SWITCHED PLATES (ALL)',
'UNKNOWN VIOLATION',
'VEHICLE BLOCKING HANDICAP CURB CUT',
'VEHICLE PARKED ON UNPAVED PORTION OR',
'CODE SECTION_Access by occupant',
'CODE SECTION_Access to Building Openings and Roofs',
'CODE SECTION_Accessibility',
'CODE SECTION_Accessory Uses',
'CODE SECTION_Administration',
'CODE SECTION_Applicability',
'CODE SECTION_Art.3 Neighborhood Zones',
'CODE SECTION_Art.6 Uses',
'CODE SECTION_Art.7 Site Development',
'CODE SECTION_Artical 6 Uses',
'CODE SECTION_Artical 9 Signs',
'CODE SECTION_Article 7 Site Development',
'CODE SECTION_Article 7 site development',
'CODE SECTION_Article 9 Signs',
'CODE SECTION_Automatic Fire Sprinkler Systems',
'CODE SECTION_BUFFALO',
'CODE SECTION_Buffalo Ordinace',
'CODE SECTION_Buffalo Ordinance',
'CODE SECTION_Build Services & Systems',
'CODE SECTION_Building Planning',
'CODE SECTION_Buisness License',
'CODE SECTION_Business License',
'CODE SECTION_Carbon Monoxide Detection',
'CODE SECTION_Carbon monoxide detection',
'CODE SECTION_Certificate of occupancy',
'CODE SECTION_Chimneys and Vents',
'CODE SECTION_Combustible wast material',
'CODE SECTION_Commercial & Recreational Vehicles',
'CODE SECTION_Component Serviceability',
'CODE SECTION_Curb Cuts and Driveways',
'CODE SECTION_Design Criteria',
'CODE SECTION_Detached House',
'CODE SECTION_Drain Pipe Materials and Sizes',
'CODE SECTION_Duct Systems',
'CODE SECTION_Duties and Powers of the Code Official',
'CODE SECTION_EQUIP / APPLIANCE LOCATION',
'CODE SECTION_EXISTING BUILDING CODE',
'CODE SECTION_EXISTING R OCCUPANCIES',
'CODE SECTION_Electical Facilities',
'CODE SECTION_Electrical Equipment',
'CODE SECTION_Electrical Facilities',
'CODE SECTION_Elevators, Escalators and Dumbwaiters',
'CODE SECTION_Emergency Measures',
'CODE SECTION_Emergency power for illumination',
'CODE SECTION_Existing non-required equipment and systems',
'CODE SECTION_Exterior Property Areas',
'CODE SECTION_Exterior Structure',
'CODE SECTION_FIRE CODE',
'CODE SECTION_Fences and Walls',
'CODE SECTION_Fire Code',
'CODE SECTION_Fire Protection Systems',
'CODE SECTION_Fire Protection and Safety',
'CODE SECTION_Fire and Smoke Protection',
'CODE SECTION_Fire-Resistance Ratings',
'CODE SECTION_Fire-Resistant Construction',
'CODE SECTION_Foundations',
'CODE SECTION_GENERAL FIRE SAFETY REQ.',
'CODE SECTION_Gen. Regs. (Condensate)',
'CODE SECTION_Gen. Regs. (Equip/Appli. locations)',
'CODE SECTION_Gen. Regs. (Repair)',
'CODE SECTION_General',
'CODE SECTION_General (Interior)',
'CODE SECTION_General Regulations',
'CODE SECTION_General Requirements',
'CODE SECTION_General requirements',
'CODE SECTION_Glazing',
'CODE SECTION_Guards and Window Fall Protection',
'CODE SECTION_Handrails',
'CODE SECTION_Handrails and Guardrails',
'CODE SECTION_Heating Facilities',
'CODE SECTION_Interior Structure',
'CODE SECTION_LANDSCAPE',
'CODE SECTION_Light',
'CODE SECTION_Light, Ventilation and Heating',
'CODE SECTION_Liquefied Petroleum Gases',
'CODE SECTION_Liquefied petroleum gases',
'CODE SECTION_Means of Egress',
'CODE SECTION_Means of Egress Illumination',
'CODE SECTION_Means of egress',
'CODE SECTION_Mechanical Equipment',
'CODE SECTION_Occupancy Limitations',
'CODE SECTION_Open Burning',
'CODE SECTION_Open Flames',
'CODE SECTION_PLUMBING CODE',
'CODE SECTION_Parking Lots',
'CODE SECTION_Pedestrain Walkways and tunnels',
'CODE SECTION_Pest Elimination',
'CODE SECTION_Plumbing Facilities and Fixture Requirements',
'CODE SECTION_Plumbing Systems and Fixtures',
'CODE SECTION_Projection of Pedestrians',
'CODE SECTION_Property Maint Supplement',
'CODE SECTION_Property Maintenance',
'CODE SECTION_Real Property Actions & Proceedings',
'CODE SECTION_Required Facilities',
'CODE SECTION_Residential',
'CODE SECTION_Roof Assemblies',
'CODE SECTION_Roof recover',
'CODE SECTION_Rubbish and Garbage',
'CODE SECTION_Safeguards during const.',
'CODE SECTION_Sanitary Drainage System',
'CODE SECTION_Sanitation',
'CODE SECTION_Scope and Application',
'CODE SECTION_Shop-Front House',
'CODE SECTION_Site Development',
'CODE SECTION_Site Devlopment',
'CODE SECTION_Site Impacts',
'CODE SECTION_Site Landscape',
'CODE SECTION_Smoke Partitions',
'CODE SECTION_Stairways',
'CODE SECTION_Storm Drainage',
'CODE SECTION_Stpo Work Order',
'CODE SECTION_Structural Design',
'CODE SECTION_Supplement',
'CODE SECTION_Supplement 2017',
'CODE SECTION_Swimming Pools, Spas and Hot Tubs',
'CODE SECTION_Swimming pools spas and hot tubs',
'CODE SECTION_Swimming pools, spas and hot tubs',
'CODE SECTION_Temporary Uses',
'CODE SECTION_The Building Code',
'CODE SECTION_Time and access',
'CODE SECTION_Toilet Rooms',
'CODE SECTION_Toilet, Bath and Shower Spaces',
'CODE SECTION_Unsafe Structures and Equioment',
'CODE SECTION_Unsafe Structures and Equipment',
'CODE SECTION_Unsafe structures and equipment',
'CODE SECTION_Unvented room heaters',
'CODE SECTION_Uses',
'CODE SECTION_V7MIGRATION',
'CODE SECTION_Ventilation',
'CODE SECTION_Venting of appliances',
'CODE SECTION_Water Heaters',
'CODE SECTION_Water Systems']]

In [0]:
y1=df_merged_parking_code['Rank']

In [0]:
from sklearn.model_selection import train_test_split
X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y1,test_size=0.3)

In [0]:
X1

,RESOLUTION_IN COMPLIANCE,RESOLUTION_PROPERTY DEMOLISHED,RESOLUTION_REFERRED TO DEMOLITION,RESOLUTION_REFERRED TO DISTRICT,RESOLUTION_UNCONDITIONAL DISCHARGE,RESOLUTION_DEFENDANT JAILED PER JUDGE,RESOLUTION_DEFENDANT FINED,RESOLUTION_CONDITIONAL DISCHARGE,RESOLUTION_CITY OF BUFFALO FORECLOSURE,RESOLUTION_CITY OF BUFFALO OWNED,RESOLUTION_CLOSE FILE REWRITE,ALTERNATE PARKING,BLOCKED DRIVEWAY (ALL),BUS RT. NO PARKING 1:30 TO 7:00 AM 11/15-4/1,DOUBLE PARKING,EXPIRED INSPECTION STICKER (ALL),EXPIRED REGISTRATION STICKER,LESS THAN 20 FT FROM CROSSWALK,LOADING ZONE,METER OVERTIME,METHOD OF PARKING(ALL),NO INSPECTIONS STICKER (ALL),NO PARK COMMERICAL TRCK ON RESIDENTIAL,NO PARKING,NO PARKING OF HOUSE TRAILERS (ALL),NO REGISTRATION STICKER (ALL),NO REPAIRS IN STREET EXCEPT EMERGENCY,NO STANDING,NO STOPPING (ALL),NO VIOLATION,ONE LICENSE PLATE MISSING (ALL),PARKED BEYOND POSTED TIME LIMIT,PARKED IN HANDICAPPED AREA,PARKED LESS THAN 15 FT - FIRE HYDRANT,"PARKED MORE THAN 12""FROM CURB",PARKED MORE THAN 72 HRS. SAME SPOT,PARKED ON SIDEWALK (ALL),PARKED OVER 18 FT FOR MORE THAN ONE,PARKING IN WRONG DIRECTION (ALL),PARKING ON DIVIDED HWYS PROHIBITED,...,CODE SECTION_Real Property Actions & Proceedings,CODE SECTION_Required Facilities,CODE SECTION_Residential,CODE SECTION_Roof Assemblies,CODE SECTION_Roof recover,CODE SECTION_Rubbish and Garbage,CODE SECTION_Safeguards during const.,CODE SECTION_Sanitary Drainage System,CODE SECTION_Sanitation,CODE SECTION_Scope and Application,CODE SECTION_Shop-Front House,CODE SECTION_Site Development,CODE SECTION_Site Devlopment,CODE SECTION_Site Impacts,CODE SECTION_Site Landscape,CODE SECTION_Smoke Partitions,CODE SECTION_Stairways,CODE SECTION_Storm Drainage,CODE SECTION_Stpo Work Order,CODE SECTION_Structural Design,CODE SECTION_Supplement,CODE SECTION_Supplement 2017,"CODE SECTION_Swimming Pools, Spas and Hot Tubs",CODE SECTION_Swimming pools spas and hot tubs,"CODE SECTION_Swimming pools, spas and hot tubs",CODE SECTION_Temporary Uses,CODE SECTION_The Building Code,CODE SECTION_Time and access,CODE SECTION_Toilet Rooms,"CODE SECTION_Toilet, Bath and Shower Spaces",CODE SECTION_Unsafe Structures and Equioment,CODE SECTION_Unsafe Structures and Equipment,CODE SECTION_Unsafe structures and equipment,CODE SECTION_Unvented room heaters,CODE SECTION_Uses,CODE SECTION_V7MIGRATION,CODE SECTION_Ventilation,CODE SECTION_Venting of appliances,CODE SECTION_Water Heaters,CODE SECTION_Water Systems
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355343,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64
355344,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor(n_estimators=600)
rfc.fit(X1_train,y1_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=600, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
y1_pred = rfc.predict(X1_test)

In [0]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

Mean Absolute Error: 1.61212336177712
Mean Squared Error: 26.998025445093678
Root Mean Squared Error: 5.195962417598272


In [0]:
pd.DataFrame(rfc.feature_importances_, X1_train.columns).sort_values(by=0).tail(10)

,0
CODE SECTION_Means of Egress Illumination,0.006175
CODE SECTION_Drain Pipe Materials and Sizes,0.006306
CODE SECTION_Open Burning,0.006357
CODE SECTION_Plumbing Facilities and Fixture Requirements,0.006386
CODE SECTION_Liquefied Petroleum Gases,0.006395
CODE SECTION_Uses,0.006628
CODE SECTION_Buffalo Ordinance,0.007313
RESOLUTION_CONDITIONAL DISCHARGE,0.012581
ALTERNATE PARKING,0.062930
RESOLUTION_DEFENDANT FINED,0.219449


In [0]:
df=pd.DataFrame({'Actual':y1_test, 'Predicted':y1_pred})
df.head(10)

,Actual,Predicted
315851,57,57.000000
27906,26,26.000000
124140,18,18.000000
235243,32,23.246206
345938,12,12.000000
223956,44,44.000000
154293,22,24.091442
76389,2,4.670887
321230,55,55.000000
297501,53,38.798904
